Get results by criteria

In [1]:
from glob import glob
import os
import json
import numpy as np
import yaml
import pandas as pd

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

output_file_name = 'quorum_output_file.yaml'
if os.path.exists(output_file_name):
    os.remove(output_file_name)

centers = {}
manufacturers = {}
strengths = {}
depths = {}
phases = {}

path_list = glob(r'quorum_output_2\temp_allClasses\*.gz')
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    center = df['Name'].iloc[0].split('-')[0]
    manufacturer = df['Manufacturer'].iloc[0]
    strength = df['Field Strength'].iloc[0]

    centers = update_dict(centers, str(center), filename)
    manufacturers = update_dict(manufacturers, str(manufacturer), filename)
    strengths = update_dict(strengths, str(strength), filename)
    phases = update_dict(phases, str(phase), filename)

with open(r'quorum_output_2\temp_allClasses\summary.json', 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    results_dict = {}
    for current_dict in [centers, manufacturers, strengths, phases]:
        for key in current_dict.keys():
            current_values = current_dict[key]
            list_of_dict = [x for x in metric_file if x['reference'].split('/')[-1] in current_values]
            mean_dice_list = []
            mean_hausdorff_list = []
            for data_dict in list_of_dict:
                mean_dice_list.append([data_dict['1']['Dice'], data_dict['2']['Dice'], data_dict['3']['Dice']])
                mean_hausdorff_list.append([data_dict['1']['Hausdorff Distance'], data_dict['2']['Hausdorff Distance'], data_dict['3']['Hausdorff Distance']])
            class_dice = np.stack(mean_dice_list, axis=0).mean(axis=0)
            class_hausdorff = np.stack(mean_hausdorff_list, axis=0).mean(axis=0)
            results_dict[key] = {'Hausdorff distance': class_hausdorff.tolist(), 
                                'Mean Hausdorff distance': float(class_hausdorff.mean()), 
                                'Dice score': class_dice.tolist(),
                                'Mean dice score': float(class_dice.mean())}

with open(output_file_name, 'w') as fd:
    yaml.safe_dump(results_dict, fd, default_flow_style=False)
    #for results_dict in results_dicts:
    #    for key in results_dict.keys():
    #        fd.write(key + ': ' + str(results_dict[key]) + '\n')
    #    fd.write('\n')


Get ED/ES volume

In [5]:
import nibabel as nib
import numpy as np

data = nib.load(r'quorum_output\validation_raw\patient003_ed.nii.gz')
zoom = data.header.get_zooms()
pixel_volume = np.prod(zoom)
print(pixel_volume)
arr = data.get_fdata()
nb_pixels = np.count_nonzero(arr == 1)
print(nb_pixels * pixel_volume)

13.234375
187504.625


In [2]:
%matplotlib qt

import nibabel as nib
import numpy as np
from scipy.ndimage import distance_transform_edt
from scipy.interpolate import interpn
import matplotlib.pyplot as plt

def bwperim(bw, n=4):
    """
    perim = bwperim(bw, n=4)
    Find the perimeter of objects in binary images.
    A pixel is part of an object perimeter if its value is one and there
    is at least one zero-valued pixel in its neighborhood.
    By default the neighborhood of a pixel is 4 nearest pixels, but
    if `n` is set to 8 the 8 nearest pixels will be considered.
    Parameters
    ----------
      bw : A black-and-white image
      n : Connectivity. Must be 4 or 8 (default: 8)
    Returns
    -------
      perim : A boolean image
    """

    if n not in (4,8):
        raise ValueError('mahotas.bwperim: n must be 4 or 8')
    rows,cols = bw.shape

    # Translate image by one pixel in all directions
    north = np.zeros((rows,cols))
    south = np.zeros((rows,cols))
    west = np.zeros((rows,cols))
    east = np.zeros((rows,cols))

    north[:-1,:] = bw[1:,:]
    south[1:,:]  = bw[:-1,:]
    west[:,:-1]  = bw[:,1:]
    east[:,1:]   = bw[:,:-1]
    idx = (north == bw) & \
          (south == bw) & \
          (west  == bw) & \
          (east  == bw)
    if n == 8:
        north_east = np.zeros((rows, cols))
        north_west = np.zeros((rows, cols))
        south_east = np.zeros((rows, cols))
        south_west = np.zeros((rows, cols))
        north_east[:-1, 1:]   = bw[1:, :-1]
        north_west[:-1, :-1] = bw[1:, 1:]
        south_east[1:, 1:]     = bw[:-1, :-1]
        south_west[1:, :-1]   = bw[:-1, 1:]
        idx &= (north_east == bw) & \
               (south_east == bw) & \
               (south_west == bw) & \
               (north_west == bw)
    return ~idx * bw

def signed_bwdist(im):
    '''
    Find perim and return masked image (signed/reversed)
    '''    
    perimeter = bwperim(im)
    distance_map = bwdist(perimeter)
    im = -distance_map*np.logical_not(im) + distance_map*im
    return im

def bwdist(im):
    '''
    Find distance map of image
    '''
    dist_im = distance_transform_edt(1-im)
    return dist_im

def interp_shape(arr, new_depth):
    '''
    Interpolate between two contours

    Input: top 
            [X,Y] - Image of top contour (mask)
           bottom
            [X,Y] - Image of bottom contour (mask)
           precision
             float  - % between the images to interpolate 
                Ex: num=0.5 - Interpolate the middle image between top and bottom image
    Output: out
            [X,Y] - Interpolated image at num (%) between top and bottom

    '''
    X, Y, Z = arr.shape

    distance_arr = []
    for i in range(Z):
        distance_arr.append(signed_bwdist(arr[:, :, i]))
    distance_arr = np.stack(distance_arr, axis=-1)

    x = np.arange(0, X)
    y = np.arange(0, Y)
    z = np.arange(0, Z)
    points = (x, y, z)

    stop = Z-1

    # create ndgrids
    grid = np.mgrid[:X, :Y, 0:stop:(new_depth * 1j)]
    xi = np.rollaxis(grid, 0, 4)
    xi = xi.reshape((X * Y * new_depth, 3))

    out = interpn(points, arr, xi)
    out = out.reshape((X, Y, new_depth))

    # Threshold distmap to values above 0
    out = out > 0

    return out


data = nib.load(r'quorum_output\validation_raw\patient003_ed.nii.gz')
arr = data.get_fdata()
arr = arr == 1

X, Y, Z = arr.shape
print(arr.shape)
# Run interpolation
out = interp_shape(arr, Z+1)
print(out.shape)
#fig, ax = plt.subplots(1, Z+1)
#for i in range(Z+1):
#    ax[i].imshow(out[:, :, i], cmap='gray')
#plt.show()

(288, 288, 13)
(288, 288, 14)


In [3]:
%matplotlib qt

import nibabel as nib
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

path_list = glob(r'quorum_output_2\temp_allClasses\*.gz')
ed_path_list = []
es_path_list = []
ed_spacing = []
es_spacing = []
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    spacing = (df['Space Between Slices'] - df['Slice Thickness']).to_numpy()[0]
    if phase == 'ed':
        ed_path_list.append(path)
        ed_spacing.append(spacing)
    elif phase == 'es':
        es_path_list.append(path)
        es_spacing.append(spacing)

#ed_path_list = [x for x in path_list if '_ed.' in x]
#es_path_list = [x for x in path_list if '_es.' in x]

results = {}
for i, class_name in enumerate(tqdm(['RV', 'MYO', 'LV']), 1):
    ed_volume_list = []
    ed_volume_list_interp = []
    for ed_path, spacing in zip(ed_path_list, ed_spacing):
        data = nib.load(ed_path)
        zoom = list(data.header.get_zooms())
        arr = data.get_fdata()
        pixel_size = np.prod(zoom)

        new_depth = round((spacing * (arr.shape[-1] - 1)) / zoom[-1]) + arr.shape[-1]
        #print(arr.shape[-1])
        #print(new_depth)
        #print('******************************')

        class_nb_pixels = arr == i

        ed_volume_list.append(pixel_size * class_nb_pixels.sum())
        arr_interpolated = interp_shape(class_nb_pixels, new_depth)
        ed_volume_list_interp.append(pixel_size * arr_interpolated.sum())

    mean_ed_volume = float(np.array(ed_volume_list).mean() / 1000)
    mean_ed_volume_interp = float(np.array(ed_volume_list_interp).mean() / 1000)

    es_volume_list = []
    es_volume_list_interp = []
    for es_path, spacing in zip(es_path_list, es_spacing):
        data = nib.load(es_path)
        zoom = list(data.header.get_zooms())
        arr = data.get_fdata()
        pixel_size = np.prod(zoom) #cm3

        new_depth = round((spacing * (arr.shape[-1] - 1)) / zoom[-1]) + arr.shape[-1]

        class_nb_pixels = arr == i

        es_volume_list.append(pixel_size * class_nb_pixels.sum())
        arr_interpolated = interp_shape(class_nb_pixels, new_depth)
        es_volume_list_interp.append(pixel_size * arr_interpolated.sum())

    mean_es_volume = float(np.array(es_volume_list).mean() / 1000)
    mean_es_volume_interp = float(np.array(es_volume_list_interp).mean() / 1000)

    results[class_name] = {'ED': {'volume': mean_ed_volume, 'interpolated_volume': mean_ed_volume_interp},
                            'ES': {'volume': mean_es_volume, 'interpolated_volume': mean_es_volume_interp}}
    if class_name != 'MYO':
        results[class_name].update({'Fraction d\'ejection': {'no_interpolation': ((mean_ed_volume - mean_es_volume) / mean_ed_volume) * 100, 'interpolated': ((mean_ed_volume_interp - mean_es_volume_interp) / mean_ed_volume_interp) * 100},
                                    'Volume d\'ejection': {'no_interpolation': (mean_ed_volume - mean_es_volume), 'interpolated': (mean_ed_volume_interp - mean_es_volume_interp)}})
                            

with open(output_file_name, 'r') as fd:
    my_dict = yaml.safe_load(fd)
    my_dict.update(results)
with open(output_file_name, 'w') as fd:
    yaml.safe_dump(my_dict, fd, default_flow_style=False)

100%|██████████| 3/3 [01:27<00:00, 29.26s/it]


Get dice per depth level

In [5]:
from evaluation.metrics import dice
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

path_list = glob(r'quorum_output_2\temp_allClasses\*.gz')
path_list_names = [x.split('\\')[-1][:13] for x in path_list]
path_list_gt = glob(r'custom_quorum_2\**\*_gt.nii.gz', recursive=True)
path_list_gt = [x for x in path_list_gt if x.split('\\')[-1][:13] in path_list_names]
assert len(path_list_gt) == len(path_list)

path_list = sorted(path_list, key=lambda x:x.split('\\')[-1])
path_list_gt = sorted(path_list_gt, key=lambda x:x.split('\\')[-1])


out_dict = {}
for path, path_gt in zip(path_list, path_list_gt):
    data = nib.load(path)
    arr = data.get_fdata()

    data_gt = nib.load(path_gt)
    arr_gt = data_gt.get_fdata()

    assert arr.shape == arr_gt.shape
    for i in range(arr.shape[-1]):
        #fig, ax = plt.subplots(1, 2)
        #ax[0].imshow(arr[:, :, i], cmap='gray')
        #ax[1].imshow(arr_gt[:, :, i], cmap='gray')
        #plt.show()
        #plt.waitforbuttonpress()
        #plt.close(fig)
        score = dice(test=arr[:, :, i], reference=arr_gt[:, :, i])
        out_dict = update_dict(out_dict, key=i/arr.shape[-1], value=score)

x = []
y = []
for key in out_dict.keys():
    x.append(key)
    y.append(np.array(out_dict[key]).mean())

fig, ax = plt.subplots(1, 1)
ax.scatter(np.array(x), np.array(y))
ax.set(xlabel='Depth as percent of volume', ylabel='Dice score')
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))